## 线性回归的简洁实现

In [1]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

In [2]:
num_features=2
num_examples=1000
true_w=[2,-2]
true_b=5.5
examples = torch.randn(num_examples,num_features,dtype=torch.float32)
labels = true_w[0]*examples[:,0]+true_w[1]*examples[:,1]+true_b
labels+=torch.tensor(np.random.normal(0,0.01,size=labels.size())
                    ,dtype=torch.float32)
labels

tensor([ 2.4655,  2.4932, 10.4394, 10.4440, 10.9183,  3.1116,  2.8337,  4.9631,
         4.2497,  5.2911,  1.9516,  8.9086,  4.6084,  3.8854, -0.2705,  4.6349,
         7.8503,  8.7553,  7.6756,  3.6597,  3.6466,  5.4732,  0.3920,  5.1678,
        -1.1402,  4.6748, 10.4345,  6.4641,  8.8305,  6.8826,  8.2196,  4.9940,
         5.9900,  5.7975,  4.2658,  3.8463,  5.9819,  7.1331,  8.3011,  3.2582,
         7.5125,  9.0564,  9.3552,  3.6004,  5.1539, 10.6855,  3.3712,  6.3470,
         8.9445,  3.7684,  5.7342,  3.1631,  8.6203,  7.5955,  1.2284,  5.4802,
         6.0713,  3.7136,  9.9302,  6.6225,  4.0045,  4.7726,  4.0944,  6.6271,
         2.1045,  4.0181,  3.0088,  8.6568,  3.8279,  8.7602,  4.8629,  5.0880,
         6.9240,  6.9412,  2.0965,  6.4297,  6.0086,  9.4800,  1.4377,  8.2213,
         5.9989,  3.6668,  5.7243,  7.3405,  6.3130,  4.9544,  6.5486,  1.1333,
         4.8474,  5.5174,  5.7575,  6.5295,  7.9151,  3.4282,  2.2480,  5.4171,
         9.5149,  8.1106,  5.9820,  2.91

In [3]:
import torch.utils.data as Data
batch_size = 10
# 将特征与标签合并
dataset = Data.TensorDataset(examples,labels)
# 随机读取小批量数据
data_iter = Data.DataLoader(dataset,batch_size,shuffle=True)

用Sequential搭建网络

In [4]:
import torch.nn as nn
net = nn.Sequential()
net.add_module('linear',nn.Linear(num_features,1))
print(net)

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


初始化模型参数

$w$采样于均值为$0$，标准差为$0.01$的正态分布，$b$为$0$

In [5]:
nn.init.normal_(net[0].weight, mean=0, std=0.01)
nn.init.constant_(net[0].bias,val=0)

Parameter containing:
tensor([0.], requires_grad=True)

使用nn自带的MSE损失函数

In [6]:
loss = nn.MSELoss()

使用torch.optim中带的SGD优化器

In [7]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(),lr=0.01)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [8]:
epochs=5
for epoch in range(1,epochs+1):
    for X,y in data_iter:
        sum_loss = loss(net(X),y.view(-1,1)) #求样本损失和
        optimizer.zero_grad()
        sum_loss.backward() #求梯度
        optimizer.step() #使用SGD优化参数
    print('epoch %d , loss %f' % (epoch,sum_loss.item()))    

epoch 1 , loss 0.573764
epoch 2 , loss 0.011898
epoch 3 , loss 0.000359
epoch 4 , loss 0.000099
epoch 5 , loss 0.000050


比较参数

In [9]:
print(net[0].weight,'\n',true_w)
print(net[0].bias,'\n',true_b)

Parameter containing:
tensor([[ 2.0004, -2.0002]], requires_grad=True) 
 [2, -2]
Parameter containing:
tensor([5.4991], requires_grad=True) 
 5.5


$torch.utils.data$模块提供了有关数据处理的工具

$torch.nn$模块定义了大量神经网络的层

$torch.nn.init$模块定义了各种初始化方法

$torch.optim$模块提供了模型参数初始化的各种方法。